In [1]:
## change venv to stripe field
import os
import veeringVideo
import veeringLogs
import importlib
import threading
from IPython.display import clear_output




In [2]:
## define root directory and day - then build folder structure
rootDirectory = '/Users/sean/mbp_storage/'
year = '24'
month = '06'
day = '09'
dayFolder = year+month+day
path_dayFolder = os.path.join(rootDirectory,dayFolder)
os.mkdir(path_dayFolder)
folders = ['aftFacing', 'fwdFacing', 'portMain', 'stbMain', 'portJib', 'stbJib', 'log']

for folder in folders:
    os.mkdir(os.path.join(path_dayFolder,folder))

FileExistsError: [Errno 17] File exists: '/Users/sean/mbp_storage/240609'

In [3]:
days = ''
days = [days.join(list(str(year))+list(str(month))+list(str(day)))]

def TL_2_JPG(rootDirectory, day, sail, filter):
    folder = os.path.join(rootDirectory,day,sail)
    for file in os.listdir(folder):
        try:
            veeringVideo.SailTimeLapse(os.path.join(folder,file),sail).TimeLapse_To_JPG()

        except:
            print("failed on  - "+str(folder+file))

        try:
            veeringVideo.Filter_JPG(os.path.join(rootDirectory,day,sail,'jpg'),filter).Remove_Files()

        except:
            print("remove failed on  - "+str(folder+file))

In [4]:
## before running paste all .mp4's into folders and manually rename dji files
# iterate over days and sails and change gopro .mp4 time laps file names to match the format YYYYMMDD_HHMMSS_timestep

gpSails = ['portMain', 'stbMain']
sails = ['portMain', 'stbMain', 'portJib', 'stbJib']
timeStep = 2
timeZone = 1
for day in days:
    for sail in gpSails:
        folder = os.path.join(rootDirectory,day,sail)
        for file in os.listdir(folder):
            try:
                veeringVideo.Rename_GP_TimeLapse(os.path.join(folder,file), timeZone, timeStep).Rename()
            except:
                print("failed on - "+str(folder+file))
        try:
            log = veeringLogs.VeeringLog(os.path.join(rootDirectory,day,'log/log.csv'))
            log.Expedition_To_DF()
            log.Select_Variables(['Utc','TWA'])
            log.Add_Time_Zone(1)
            log.Add_Time_Stamp()
            log.Get_Filter_TS(30,60)
            filters = [log.upwindPort,log.upwindStb,log.upwindPort, log.upwindStb]
            
        except:
            print("Log Failed")

    thrds = []
    for i in range(len(sails)):
        sail = sails[i]
        filter = filters[i]
        thrds.append(threading.Thread(target=TL_2_JPG, args=(rootDirectory, day, sail, filter)))

    thrds[0].start()
    thrds[1].start()
    thrds[2].start()
    thrds[3].start()
    
        

remove failed on  - /Users/sean/mbp_storage/240609/portJib.DS_Store
remove failed on  - /Users/sean/mbp_storage/240609/stbJib.DS_Store


In [14]:
runningCheck = []
for i in range(len(thrds)):
    runningCheck.append(thrds[i].is_alive())
any(runningCheck)

False

In [5]:
## iterate over days and sails and convert .mp4 time laps to a folder of .jpg. jpg folder is created.            
thrds = []
for i in range(len(sails)):
    sail = sails[i]
    filter = filters[i]
    thrds.append(threading.Thread(target=TL_2_JPG, args=(rootDirectory, day, sail, filter)))

thrds[0].start()
thrds[1].start()
thrds[2].start()
thrds[3].start()




In [58]:
sails = ['portMain', 'stbMain', 'portJib', 'stbJib']

for day in days:
    log = veeringLogs.VeeringLog(os.path.join(rootDirectory,day,'log/log.csv'))
    log.Expedition_To_DF()
    log.Select_Variables(['Utc','TWA'])
    log.Add_Time_Zone(1)
    log.Add_Time_Stamp()
    log.Get_Filter_TS(30,60)
    filters = [log.upwindPort,log.upwindStb,log.upwindPort, log.upwindStb]

    for i in range(len(filters)):
        sail = sails[i]
        filter = filters[i]
        try:
            veeringVideo.Filter_JPG(os.path.join(rootDirectory,day,sail,'jpg'),filter).Remove_Files()
        except:
            print("fail on file - "+str(os.path.join(rootDirectory,day,sail,'jpg')))
